In [156]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import subprocess
import time
import numpy as np
from tqdm import notebook

In [2]:
# headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

# url = f"https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%EC%9D%98_%EC%8B%AC%EC%95%BC%EB%B2%84%EC%8A%A4_%EB%85%B8%EC%84%A0_%EB%AA%A9%EB%A1%9D"

# result = requests.get(url, headers=headers)
# soup = BeautifulSoup(result.text, "html.parser")  # 페이지 불러와서 bs4로 변경

# k = soup.find("table", "wikitable")  # 페이지 별 최신 날짜 찾기


> ### selenium setting

In [3]:
subprocess.Popen(
    'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="D:\chromdriver\Temp"'
)
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
chrome_driver = "D:\chromdriver\chromedriver.exe"
browser = webdriver.Chrome(chrome_driver, options=chrome_options)
browser.implicitly_wait(time_to_wait=7)


C:\Users\679oo\AppData\Local\Temp/ipykernel_26888/2094588.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_driver, options=chrome_options)


> ### bus_info_scrap

In [ ]:
a = {'N13' : ['상계주공7단지','복정역환승센터}'],
'N15' : ['우이동','사당역}'],
'N16' : ['도봉차고지','온수동차고지}'],
'N26' : ['강서공영차고지','중랑공영차고지}'],
"N30" : [' 강동공영차고지','서울역환승센터'],
"N32" : ['송파공영차고지','신설동역'],
"N34" : ['강동공영차고지','신사역'],
"N37" : ['복정역환승센터','진관공영차고지'],
"N51" : ['시흥동차고지','하계동차고지'],
"N61" : ['양천공영차고지','상계주공7단지'],
"N62" : ['양천공영차고지','면목동차고지'],
"N64" : ['강서공영차고지','염곡공영차고지'],
"N72" : ['은평공영차고지','신설동역'],
"N75" : ['진관공영차고지','신림2동차고지']}

In [87]:
bus_names = [
    "N13",
    "N15",
    "N16",
    "N26",
    "N30",
    "N32",
    "N34",
    "N37",
    "N51",
    "N61",
    "N62",
    "N64",
    "N72",
    "N75",
]
result = []
for bus_name in bus_names:
    # 검색
    browser.get("https://bus.go.kr/searchResult6.jsp")
    searching = browser.find_element(By.ID, "searchname")
    searching.send_keys(f"{bus_name}")
    time.sleep(0.1)
    searching.send_keys(Keys.ENTER)
    time.sleep(0.5)

    # irfame 이동 후 scraping
    browser.switch_to.frame("introflash")
    html1 = BeautifulSoup(browser.page_source, "html.parser")
    st_id = [a.text for a in html1.find_all("text", "stationNo")]
    st_name = [a.text for a in html1.find_all("text", "stationName")]
    # dataframe만들기
    st_id_dt = pd.DataFrame([st_id, st_name]).T
    st_id_dt.columns = ["id", "name"]
    # st_id_dt["bus"] = f"{bus_name}"

    result.append(st_id_dt)


final = pd.concat(result, keys=bus_names, axis=0)

# 1873 rows × 2 columns
final_copy = final.sort_values(by="id").iloc[21:]

final_copy["id"] = final_copy["id"].astype(int)

final_copy = final_copy.reset_index(drop=False)

id          name
N13 0    11193  온곡중학교노원정보도서관
    1    11208        상계초등학교
    2    11210       노원역9번출구
    3    11244       상계주공3단지
    4    11245       상계주공2단지
...        ...           ...
N75 168  12013   박석고개.신도고등학교
    169              구파발(가상)
    170  12111      구파발역2번출구
    171  12009     은평뉴타운10단지
    172  12470       진관공영차고지

[1873 rows x 2 columns]

> ### Loading master_bus_info

In [131]:
master_bus = pd.read_csv("assets/서울시 정류장마스터 정보.csv", encoding="CP949")

# remove_row = [48899,47206,47022,42811,38961,39246,39248,38326,38329,38244,38245,34215,34099,31924,31899,30741,30742,30811,30812]

master_bus.drop(index=remove_row, inplace=True)



> ### Merging scraping_bus_info with master_bust_info

In [138]:
night_bus_info = pd.merge(
    final_copy,
    master_bus,
    how="left",
    left_on=["id", "name"],
    right_on=["정류장_번호", "정류장_명칭"],
    ).drop(columns=["정류장_번호", "버스도착정보안내기_설치_여부", "정류장_ID", "정류장_유형", "정류장_명칭"])

night_bus_info = (
    night_bus_info.set_index(["level_0", "level_1"])
    .sort_values(by=["level_0", "level_1"])
    .reset_index(drop=False)
)
night_bus_info.columns = ["bus", "order", "id", "name", "경도", "위도"]

In [139]:
night_bus_info


,bus,order,id,name,경도,위도
0,N13,0,11193,온곡중학교노원정보도서관,127.064393,37.659201
1,N13,1,11208,상계초등학교,127.065774,37.657587
2,N13,2,11210,노원역9번출구,127.061986,37.656087
3,N13,3,11244,상계주공3단지,127.060908,37.652096
4,N13,4,11245,상계주공2단지,127.061620,37.649637
...,...,...,...,...,...,...
1847,N75,167,12015,세명컴퓨터고등학교,126.919973,37.624215
1848,N75,168,12013,박석고개.신도고등학교,126.919374,37.629919
1849,N75,170,12111,구파발역2번출구,126.918989,37.636768
1850,N75,171,12009,은평뉴타운10단지,126.915165,37.642948


In [113]:
# a.to_csv('assets/bus_station_finish.csv',encoding="CP949")


> ### finding bike stations near the night bus station

In [149]:
night_bus_station = pd.read_csv('assets/night_bus_station.csv',encoding="CP949",index_col=0)
station = pd.read_csv('assets/seoul_bike_station_01_12.csv',encoding="CP949",index_col=0)

In [202]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m


bike_lat = station["위도"]
bike_long = station["경도"]

temp_save = []
for i in notebook.tqdm(range(len(night_bus_station))):
    # for i in range(10) :
    
    bus_lat = night_bus_station["위도"][i]
    bus_long = night_bus_station["경도"][i]
    result_km = haversine_np(bike_long, bike_lat, bus_long, bus_lat)
    bus_index = pd.concat([station, result_km], axis=1)
    bus_index.rename(columns={0: "거리"}, inplace=True)


    try: # 반경 400m 자전거 존으로 생각
        bus_index = bus_index.query("거리 < 200")["id"].values
    except:
        bus_index = 0
    temp_save.append(bus_index)

near_bus = pd.DataFrame([temp_save]).T
near_bus["bus_id"] = night_bus_station["id"]
near_bus["bus_name"] = night_bus_station["name"]
near_bus["경도"] = night_bus_station["경도"]
near_bus["위도"] = night_bus_station["위도"]
near_bus.rename(inplace=True, columns={0: "bike_id"})
near_bus

  0%|          | 0/1852 [00:00<?, ?it/s]

,bike_id,bus_id,bus_name,경도,위도
0,[1691],11193,온곡중학교노원정보도서관,127.064393,37.659201
1,[4003],11208,상계초등학교,127.065774,37.657587
2,"[1637, 1653]",11210,노원역9번출구,127.061986,37.656087
3,[],11244,상계주공3단지,127.060908,37.652096
4,[1636],11245,상계주공2단지,127.061620,37.649637
...,...,...,...,...,...
1847,[947],12015,세명컴퓨터고등학교,126.919973,37.624215
1848,"[921, 993]",12013,박석고개.신도고등학교,126.919374,37.629919
1849,"[905, 960]",12111,구파발역2번출구,126.918989,37.636768
1850,[959],12009,은평뉴타운10단지,126.915165,37.642948


> ### Saving results

In [203]:
near_bus.to_csv('assets/near_bus_200m.csv',encoding="CP949")